# Loading data and working with grandPy objects

<span style="color:green">Notiz: alles was rot ist, ist noch nicht umgesetzt oder müsste noch überarbeitet werden durch Links etc.</span>

GrandPy is a Python package for the analysis of RNA-seq experiments involving metabolic RNA labeling with nucleotide conversion, such as SLAM-seq experiments <span style="color:red">[https://www.nature.com/articles/nmeth.4435]</span>. In such experiments, nucleoside analogs such as 4sU are added to living cells, which take it up and incorporate it into newly synthesized RNA. Before sequencing, 4sU is converted into a cytosin analog. Reads covering 4sU sites therefore have characteristic T-to-C mismatches after read mapping, in principle providing the opportunity to differentiate newly synthesized (during the time of labeling) from preexisting RNA.

Confounders such as sequencing errors or read that originate from newly synthesized RNA but, by chance, do not cover sites of 4sU incorporation (usually 20-80% of all "new read") can be handled using specialized methods such as GRAND-SLAM <span style="color:red">[https://academic.oup.com/bioinformatics/article/34/13/i218/5045735?login=true]</span>.

# Reading in the data

Throughout this vignette, we will be using the GRAND-SLAM processed SLAM-seq data set from Finkel et al.2021 <span style="color:red">[https://academic.oup.com/bioinformatics/article/34/13/i218/5045735]</span>. The data set contains time series (progressive labeling) samples from a human epithelial cell line (Calu3 cells); half of the samples were infected with SARS-CoV-2 for different periods of time.

The output of GRAND-SLAM is a tsv file where rows are genes and columns are read counts and other statistics (e.g., the new-to-total RNA ratio) for all samples. The data set is available on zenodo ("https://zenodo.org/record/5834034/files/sars.tsv.gz"). We start by reading this file into Python:

In [1]:
# Package installation
from Py.load import *
sars = read_grand("../data/sars.tsv", design=("Condition", "Time", "Replicate"))
print(sars.columns)

Detected dense format -> using dense reader
['Mock.no4sU.A', 'Mock.1h.A', 'Mock.2h.A', 'Mock.2h.B', 'Mock.3h.A', 'Mock.4h.A', 'SARS.no4sU.A', 'SARS.1h.A', 'SARS.2h.A', 'SARS.2h.B', 'SARS.3h.A', 'SARS.4h.A']


C:\Users\Kathi\PycharmProjects\grandpy\Py\utils.py:61: UserWarning: Duplicate gene symbols found: STPG4, PDE11A, TXNRD3NB, ARL14EPL, HIST1H3D, SOGA3, MATR3, RABGEF1, POLR2J2, KBTBD11-OT1; they have been renamed to ensure uniqueness (e.g., MATR3 → MATR3_1).
  warnings.warn(f"Duplicate gene symbols found: {', '.join(duplicates_list)}; they have been renamed to ensure uniqueness (e.g., MATR3 → MATR3_1).")


When reading in the file, we have to define the ``design`` vector. This is used to infer metadata automatically from sample names. Here sample names consist of three parts separated by dots as shown above (the Columns function returns the sample names or cell ids when analyzing a single cell data set). Each part in the sample name represents an aspect of the design. For example, the sample named Mock.2h.A is a sample from the mock condition (i.e. not infected by SARS-CoV-2), subjected to metabolic labeling for 2 hours, and is the first replicate (i.e. replicate "A"). This sample name is consistent with the three element design vector used above. It is possible to specify other design elements (of course the samples would have to be named accordingly). A list of reasonable options is predefined in the dictionary `DESIGN_KEYS`.

There are names (i.e. the things you specify in the design vector) that have additional semantics. For example, for the name `duration.4sU`the values are interpreted like this: 4h is converted into the number 4, 30min into 0.5, and no4sU into <span style="color:red">0 (bei uns gerade NaN)</span>. For more information, see <span style="color:red">below</span>. The design vector is mandatory. Attempting to read in the data without it results in an error:

In [2]:
sars_wrong = read_grand("../data/sars.tsv") # Fehlermeldung könnte man noch verkürzen, gerade schon dolle lang

Detected dense format -> using dense reader


C:\Users\Kathi\PycharmProjects\grandpy\Py\utils.py:61: UserWarning: Duplicate gene symbols found: STPG4, PDE11A, TXNRD3NB, ARL14EPL, HIST1H3D, SOGA3, MATR3, RABGEF1, POLR2J2, KBTBD11-OT1; they have been renamed to ensure uniqueness (e.g., MATR3 → MATR3_1).
  warnings.warn(f"Duplicate gene symbols found: {', '.join(duplicates_list)}; they have been renamed to ensure uniqueness (e.g., MATR3 → MATR3_1).")


ValueError: Design must be  explicitly provided.

Alternatively, a table containing the metadata can be specified. Make sure that it contains a `Name` column matching the names in the GRAND-SLAM output table:

In [7]:
import pandas as pd
name = ["Mock.no4sU.A", "Mock.1h.A", "Mock.2h.A", "Mock.2h.B", "Mock.3h.A", "Mock.4h.A", "SARS.no4sU.A", "SARS.1h.A", "SARS.2h.A", "SARS.2h.B", "SARS.3h.A", "SARS.4h.A"]

conditions = ["Mock"] * 6 + ["SARS"] * 6

design_df = pd.DataFrame({"Name": name,
                          "Condition": conditions})

sars_meta = read_grand("../data/sars.tsv", design = design_df)

Detected dense format -> using dense reader


C:\Users\Kathi\PycharmProjects\grandpy\Py\utils.py:61: UserWarning: Duplicate gene symbols found: STPG4, PDE11A, TXNRD3NB, ARL14EPL, HIST1H3D, SOGA3, MATR3, RABGEF1, POLR2J2, KBTBD11-OT1; they have been renamed to ensure uniqueness (e.g., MATR3 → MATR3_1).
  warnings.warn(f"Duplicate gene symbols found: {', '.join(duplicates_list)}; they have been renamed to ensure uniqueness (e.g., MATR3 → MATR3_1).")


# What is the grandPy object

`read_grand` returns a grandPy object, which contains

1. metadata for genes
2. metadata for samples/cells (as inferred from the sample names by the design parameter)
3. all data matrices (counts, normalized counts, ntrs, etc. these types of data are called "slots")
4. analysis results

Metadata (1. and 2.) are described below. How to work with the data matrices and analysis results is described in a separate <span style="color:red">vignette [Working with data matrices and analysis results]</span>.

# Working with grandPy objects

Here we will see how to work with grandPy objects in general. A short summary can be displayed when `print`ing the object, and there are several functinos to retrieve general information about the object:

In [9]:
print(sars) # Link fehlt

GrandPy:
Read from sars
19659 genes, 12 samples/cells
Available data slots: ['count', 'ntr', 'alpha', 'beta']
Available analyses: []
Available plots: {}
Default data slot: count



In [10]:
print(sars.title)

sars


In [13]:
print(len(sars.genes)) # so?

19659


In [14]:
print(len(sars.coldata)) # so?

12


It is straight-forward to filter genes:

In [16]:
# sars = sars.filter_genes # fehlt bei processing.py
# print(len(sars.genes))

By default genes are retained if they have 100 read counts in at least half of the samples (or cells). There are many options how to filter by genes (note that `filter_genes` returns a new grandPy object, and below we directly call `len()` on this new object to check how many genes are retained by filtering):

In [18]:
# print("Genes with at least 1000 read counts in half of the columns: \n", len(sars.genes)) # filter_genes fehlt bei processing.py

In [19]:
# print("Genes with at least 1000 read counts in half of the columns (retain two genes that are otherwise filtered): \n", ) # filter_genes fehlt bei processing.py

In [20]:
# print("Keep only these two genes: \n", ...) # filter_genes fehlt bei processing.py

In [21]:
# sars = sars.normalizeTPM # normalizeTPM() fehlt bei processing.py
# print("Genes with at least 10 TPM in half of the columns: \n", ...)

`filter_genes()` essentially removes rows from the data slots. It is also possible to remove columns (i.e. samples or cells). This is done using the subset function:

In [24]:
mock = sars[:, sars.coldata["Condition"] == "Mock"]
print(mock) # filter_genes() fehlt noch, daher genes Anzhal noch falsch aber samples/cells richtig

GrandPy:
Read from sars
19659 genes, 6 samples/cells
Available data slots: ['count', 'ntr', 'alpha', 'beta']
Available analyses: []
Available plots: {}
Default data slot: count



The new grandPy object now only has 6 columns. The `columns`parameter to subset must be a logical vector, and you can use the names of the column metadata table (see below) as variables (i.e. the parameter here is a logical vector with all samples being TRUE where the `Condition`column is equal to "Mock".

A closely related function is `split`, which returns a list of several grandPy objects, each composed of samples having the same `Condition`.

In [25]:
# .split() fehlt in grandpy.py

In [26]:
# lapply - funktion raussuchen in python

The inverse of `split` is merge: